In [1]:
import gensim
import nltk
from gensim import corpora,models, similarities
import json
import numpy as np
import keras
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dropout
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.utils import normalize
import re
import pandas as pd

Using TensorFlow backend.


In [2]:
cornel = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [3]:
id2line = {}
for line in cornel:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [4]:
conv = []
for line in conversations:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conv.append(_line.split(','))

In [5]:
questions = []
answers = []

for con in conv:
    for i in range(len(con)-1):
        questions.append(id2line[con[i]])
        answers.append(id2line[con[i+1]])

In [6]:
data = pd.read_json('conversation.json')

data = data['conversations']

In [7]:
lenth = 300

In [8]:
x = []; y=[]


for i in range(len(data)):
    for j in range(len(data[i])):
        if j < len(data[i])-1:
            x.append(data[i][j])
            y.append(data[i][j+1])



In [9]:
for i in range(len(questions)):
    x.append(questions[i])

In [10]:
for i in range(len(answers)):
    y.append(answers[i])

In [11]:
corpus = x + y

In [13]:
raw_text = u""
for sentence in corpus:
    raw_text+=sentence
    raw_text+= " "

In [12]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [14]:
raw_corpus = tokenizer.tokenize(str(corpus))

In [15]:
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]", " ", raw)
    words = clean.split()
    return words

In [16]:
sentences = []
for sent in raw_corpus:
    if len(sent) > 0:
        sentences.append(sentence_to_wordlist(sent.lower()))

In [28]:
# sentences[:10]

In [32]:
tok_x = []
tok_y = []

for i in range(len(x)):
    tok_x.append(nltk.word_tokenize(x[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))

In [17]:
sent_end = np.ones((300,), dtype=np.float32)

In [18]:
token_x = tokenizer.tokenize(str(x))
token_y = tokenizer.tokenize(str(y))

In [19]:
sent_x = []
sent_y = []
for words in token_x:
    if len(words) > 0:
        sent_x.append(np.array(sentence_to_wordlist(words.lower())))

for wordy in token_y:
    if len(wordy) > 0:
        sent_y.append(np.array(sentence_to_wordlist(wordy.lower())))   

In [19]:
#model = gensim.models.KeyedVectors.load_word2vec_format('/media/rootmedia/modules/GoogleNews-vectors-negative300.bin/data', binary=True)

# Build word2vec model

In [20]:
#defining hyperparameters
import multiprocessing

#dimensionality of the resulting vector
num_features = 300

#minimum word count 
min_word_count = 1

#cpus to run in parallel
num_workers = multiprocessing.cpu_count()

#context window size
context_size = 5

#downsample for frequent words
down_sampling = 1e-3

#seed for RNG, to make result reproducible
seed = 1

In [21]:
chat_word2vec = gensim.models.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=down_sampling
)

In [22]:
chat_word2vec.build_vocab(sentences=sentences)
chat_word2vec.train(sentences, total_examples=chat_word2vec.corpus_count, epochs=chat_word2vec.epochs)

(17282222, 24515695)

In [23]:
X = []
Y = []
for sent in sent_x:
    sent_vec = [chat_word2vec[w] for w in sent]
    X.append(sent_vec)
    
for sent in sent_y:
    sent_v = [chat_word2vec[w] for w in sent] 
    Y.append(sent_v) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [26]:
for tok_sent in X:
    tok_sent[14:]=[]
    tok_sent.append(sent_end)
    pass

for tok_sent in X:
    if len(tok_sent) < 15:
        for i in range(15 - len(tok_sent)):
            tok_sent.append(sent_end)
            pass
        pass
    pass

for tok_sent in Y:
    tok_sent[14:]=[]
    tok_sent.append(sent_end)
    pass

for tok_sent in Y:
    if len(tok_sent) < 15:
        for i in range(15 - len(tok_sent)):
            tok_sent.append(sent_end)
            pass
        pass
    pass
X = X[:300000]
Y = Y[:300000]

In [27]:
vec_x = np.array(X)
vec_y = np.array(Y)

MemoryError: 

In [49]:
v

MemoryError: 

In [47]:
vec_x.shape, vec_y.shape

NameError: name 'vec_x' is not defined

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)
shape =train_x.shape[1:]

In [ ]:
chat = Sequential()

chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(Dropout(0.2))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))
chat.add(LSTM(output_dim=lenth,input_shape=shape, return_sequences=True,  activation='sigmoid'))


chat.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

In [ ]:
chat.summary()

In [451]:
chat.fit(vec_x, vec_y,validation_data=(test_x, test_y), epochs=10000, batch_size=10, verbose=2)

Train on 75 samples, validate on 15 samples
Epoch 1/10
 - 37s - loss: -5.9031e-01 - acc: 0.0089 - val_loss: -5.4402e-01 - val_acc: 0.0311
Epoch 2/10
 - 14s - loss: -5.9684e-01 - acc: 0.0258 - val_loss: -5.4822e-01 - val_acc: 0.0356
Epoch 3/10
 - 14s - loss: -6.0163e-01 - acc: 0.0249 - val_loss: -5.5208e-01 - val_acc: 0.0222
Epoch 4/10
 - 14s - loss: -6.0574e-01 - acc: 0.0204 - val_loss: -5.5566e-01 - val_acc: 0.0311
Epoch 5/10
 - 14s - loss: -6.0943e-01 - acc: 0.0276 - val_loss: -5.6086e-01 - val_acc: 0.0356
Epoch 6/10
 - 14s - loss: -6.1460e-01 - acc: 0.0373 - val_loss: -5.6753e-01 - val_acc: 0.0356
Epoch 7/10
 - 15s - loss: -6.2328e-01 - acc: 0.0267 - val_loss: -5.8159e-01 - val_acc: 0.0311
Epoch 8/10
 - 14s - loss: -6.3359e-01 - acc: 0.0347 - val_loss: -5.8556e-01 - val_acc: 0.0444
Epoch 9/10
 - 14s - loss: -6.3722e-01 - acc: 0.0302 - val_loss: -5.9073e-01 - val_acc: 0.0400
Epoch 10/10
 - 14s - loss: -6.3880e-01 - acc: 0.0284 - val_loss: -5.9197e-01 - val_acc: 0.0311


In [429]:
def convert(text):
    text_sent = nltk.word_tokenize(text.lower())
    vec_sent = [model[w] for w in text_sent if w in model.vocab]
    
    vec_sent[14:]=[]
    vec_sent.append(sent_end)
    if len(vec_sent) < 15:
        for i in range(15 - len(vec_sent)):
            vec_sent.append(sent_end)
            
    vec_sent = np.array([vec_sent])
    
    return vec_sent

In [458]:
message = x[31]
print(message)

Alice in Wonderland


In [459]:
process_sent = convert(message)

In [460]:
predictions = chat.predict(process_sent)

In [462]:
predictions

array([[[0.        , 0.6822104 , 0.02886251, ..., 0.09994563,
         0.        , 0.52483565],
        [0.        , 0.78655446, 0.01924976, ..., 0.04547317,
         0.        , 0.5779651 ],
        [0.        , 0.8051806 , 0.07642208, ..., 0.08082875,
         0.        , 0.523006  ],
        ...,
        [0.30746734, 0.3338932 , 0.30343533, ..., 0.31324923,
         0.30551645, 0.31196627],
        [0.3113735 , 0.33270657, 0.30430123, ..., 0.31539944,
         0.30744475, 0.31161708],
        [0.31426823, 0.331871  , 0.30490828, ..., 0.3170346 ,
         0.30878586, 0.31137162]]], dtype=float32)

In [463]:
outputlist = [chat_word2vec.most_similar([predictions[0][i]])[0][0] for i in range(15)]
output = " ".join(outputlist)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [464]:
print(output)

i i i i i not not not not not not not not not not


In [48]:
#raw_text

In [27]:
# 

In [12]:
#

In [13]:
#raw_corpus

In [29]:
# #X, Y


In [43]:
# sent_x = np.array(sent_x)
# sent_y = np.array(sent_y)

(80,)

In [289]:
# sent_x

(2956, 5235)

In [322]:
chat_word2vec.similar_by_word("good")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('masked', 0.1910114586353302),
 ('silenced', 0.14265061914920807),
 ('baking', 0.11829502135515213),
 ('often', 0.1181270033121109),
 ('activity', 0.10992403328418732),
 ('fond', 0.10934004932641983),
 ('mad', 0.09406562894582748),
 ('has', 0.08900481462478638),
 ('counts', 0.08638014644384384),
 ('you', 0.08545640856027603)]

In [45]:
X = []

In [46]:
Y = []

In [1]:
# X[0]

In [49]:

    
# for tok_sent in Y:
#     tok_sent[4:]=[]
#     tok_sent.append(sent_end)    

In [53]:
# train_x = X[:70]
# train_y = Y[:70]

In [66]:
# shape = (5, 300)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(5, 300), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(5, 300), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(5, 300), return_sequences=True, activation="relu", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  """
/usr/local/lib/python3.6/dist-packages

In [68]:
# chat.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 5, 300)            721200    
_________________________________________________________________
lstm_8 (LSTM)                (None, 5, 300)            721200    
_________________________________________________________________
lstm_9 (LSTM)                (None, 5, 300)            721200    
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 300)            721200    
Total params: 2,884,800
Trainable params: 2,884,800
Non-trainable params: 0
_________________________________________________________________


In [70]:
# chat.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=10, verbose=2)

ValueError: Error when checking input: expected lstm_7_input to have 3 dimensions, but got array with shape (70, 1)

In [216]:
# sentence = "how are you doing"

In [217]:
# pred_x = tokenizer.tokenize(str(sentence))

In [218]:
# pred_token = []

# for words in pred_x:
#     if len(words) > 0:
#         pred_token.append(sentence_to_wordlist(words))


In [219]:
# pred_token

[['how', 'are', 'you', 'doing']]

In [252]:
# X_pred = []
# for sent in pred_token:
#     sent_vec = [chat_word2vec[w] for w in sent ]
#     X_pred.append(sent_vec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [221]:
# for tok_sent in X_pred:
#     tok_sent[4:]=[]
#     tok_sent.append(sent_end)

In [237]:
# X_pred = np.array(X_pred)

In [240]:
# pred = chat.predict(X_pred)

In [249]:
outputword = [chat_word2vec.most_similar([pred[0][i]])[0][0] for i in range(5)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [250]:
# outputword

['I', 't', 's', 'C', 'C']

In [248]:
output = " ".join(outputword)

TypeError: sequence item 0: expected str instance, tuple found

In [246]:
# output

'I t s C C'

In [34]:
#model = gensim.models.Word2Vec(token_corpus, min_count=1, size=15)